# Focused Learning: Reasoning vs Non-Reasoning Model Performance Analysis

## Learning Objectives
1. Understand the fundamental differences between reasoning and non-reasoning models
2. Learn to analyze performance patterns across different problem types
3. Master techniques for evaluating long-form reasoning in code generation
4. Implement metrics for measuring reasoning quality and effectiveness

## Concept Source
- **Paper Section**: Section 3 (Holistic Evaluation) and Table 2 (Model pass rates by difficulty)
- **Key Table**: Table 3 (Pass rates across topic tags)
- **Critical Finding**: "The evaluation highlights DeepSeek-R1 (pass@1 rate = 65.23%) and QwQ-Plus (pass@1 rate = 56.25%) as top performers, demonstrating the substantial advantage of long-CoT reasoning models."

## 1. Understanding Reasoning vs Non-Reasoning Models

### What Makes a Model "Reasoning"?

**Reasoning Models** (DeepSeek-R1, QwQ-Plus):
- Use **Chain-of-Thought (CoT)** reasoning
- Generate explicit intermediate steps
- Can revise and correct their approach
- Show their "thinking" process

**Non-Reasoning Models** (GPT-4o, Claude-3.7-Sonnet, DeepSeek-V3):
- Generate solutions directly
- Rely on pattern matching and memorization
- Less explicit about problem-solving steps
- Faster but potentially less reliable for complex problems

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Optional
import json
from dataclasses import dataclass
from scipy import stats
import re
from collections import defaultdict

# Set up visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Paper's Performance Data Analysis

Let's recreate and analyze the performance data from the paper:

In [ ]:
@dataclass
class ModelPerformance:
    """Store model performance data"""
    name: str
    model_type: str  # 'reasoning' or 'non_reasoning'
    easy: float
    medium: float
    hard: float
    overall: float
    release_date: str
    
class ReasoningAnalyzer:
    """Analyze reasoning vs non-reasoning model performance"""
    
    def __init__(self):
        # Performance data from Table 2 in the paper
        self.model_data = [
            ModelPerformance("GPT-4o-0806", "non_reasoning", 81.48, 32.76, 10.47, 35.55, "2024-08"),
            ModelPerformance("Claude-3.7-Sonnet", "non_reasoning", 87.04, 54.31, 23.26, 50.78, "2024-06"),
            ModelPerformance("DeepSeek-V3", "non_reasoning", 77.78, 31.90, 13.95, 35.55, "2024-12"),
            ModelPerformance("DeepSeek-R1", "reasoning", 94.44, 68.97, 41.86, 65.23, "2025-01"),
            ModelPerformance("Qwen2.5-Max", "non_reasoning", 74.07, 25.00, 10.47, 30.47, "2024-11"),
            ModelPerformance("QwQ-Plus", "reasoning", 92.59, 62.93, 24.42, 56.25, "2024-12")
        ]
        
        # Topic-wise performance from Table 3 (subset for analysis)
        self.topic_data = {
            'Array': {'GPT-4o': 32.1, 'DeepSeek-V3': 34.5, 'Claude-3.7': 51.2, 'DeepSeek-R1': 67.9, 'QwQ-Plus': 55.4},
            'Dynamic Programming': {'GPT-4o': 10.5, 'DeepSeek-V3': 15.8, 'Claude-3.7': 31.6, 'DeepSeek-R1': 70.2, 'QwQ-Plus': 40.4},
            'Binary Search': {'GPT-4o': 7.7, 'DeepSeek-V3': 23.1, 'Claude-3.7': 30.8, 'DeepSeek-R1': 73.1, 'QwQ-Plus': 30.8},
            'Tree': {'GPT-4o': 27.3, 'DeepSeek-V3': 18.2, 'Claude-3.7': 9.1, 'DeepSeek-R1': 72.7, 'QwQ-Plus': 54.5},
            'Graph': {'GPT-4o': 40.0, 'DeepSeek-V3': 33.3, 'Claude-3.7': 53.3, 'DeepSeek-R1': 66.7, 'QwQ-Plus': 66.7},
            'Math': {'GPT-4o': 38.2, 'DeepSeek-V3': 40.0, 'Claude-3.7': 56.4, 'DeepSeek-R1': 69.1, 'QwQ-Plus': 58.2},
            'Greedy': {'GPT-4o': 12.5, 'DeepSeek-V3': 15.6, 'Claude-3.7': 21.9, 'DeepSeek-R1': 62.5, 'QwQ-Plus': 28.1},
            'Simulation': {'GPT-4o': 63.2, 'DeepSeek-V3': 57.9, 'Claude-3.7': 63.2, 'DeepSeek-R1': 63.2, 'QwQ-Plus': 84.2}
        }
    
    def create_performance_dataframe(self) -> pd.DataFrame:
        """Convert model data to DataFrame for analysis"""
        data = []
        for model in self.model_data:
            data.append({
                'model': model.name,
                'type': model.model_type,
                'easy': model.easy,
                'medium': model.medium,
                'hard': model.hard,
                'overall': model.overall,
                'release_date': model.release_date
            })
        return pd.DataFrame(data)
    
    def analyze_difficulty_patterns(self, df: pd.DataFrame) -> Dict:
        """Analyze performance patterns across difficulty levels"""
        reasoning_models = df[df['type'] == 'reasoning']
        non_reasoning_models = df[df['type'] == 'non_reasoning']
        
        analysis = {
            'reasoning_avg': {
                'easy': reasoning_models['easy'].mean(),
                'medium': reasoning_models['medium'].mean(),
                'hard': reasoning_models['hard'].mean(),
                'overall': reasoning_models['overall'].mean()
            },
            'non_reasoning_avg': {
                'easy': non_reasoning_models['easy'].mean(),
                'medium': non_reasoning_models['medium'].mean(),
                'hard': non_reasoning_models['hard'].mean(),
                'overall': non_reasoning_models['overall'].mean()
            }
        }
        
        # Calculate improvement ratios
        analysis['improvement_ratios'] = {
            'easy': analysis['reasoning_avg']['easy'] / analysis['non_reasoning_avg']['easy'],
            'medium': analysis['reasoning_avg']['medium'] / analysis['non_reasoning_avg']['medium'],
            'hard': analysis['reasoning_avg']['hard'] / analysis['non_reasoning_avg']['hard'],
            'overall': analysis['reasoning_avg']['overall'] / analysis['non_reasoning_avg']['overall']
        }
        
        return analysis
    
    def create_difficulty_visualization(self, df: pd.DataFrame):
        """Create visualization of difficulty-based performance"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Plot 1: Individual model performance
        reasoning_models = df[df['type'] == 'reasoning']
        non_reasoning_models = df[df['type'] == 'non_reasoning']
        
        x_pos = np.arange(len(df))
        
        # Create grouped bar chart
        width = 0.2
        
        bars1 = ax1.bar(x_pos - 1.5*width, df['easy'], width, label='Easy', alpha=0.8)
        bars2 = ax1.bar(x_pos - 0.5*width, df['medium'], width, label='Medium', alpha=0.8)
        bars3 = ax1.bar(x_pos + 0.5*width, df['hard'], width, label='Hard', alpha=0.8)
        bars4 = ax1.bar(x_pos + 1.5*width, df['overall'], width, label='Overall', alpha=0.8)
        
        # Color reasoning models differently
        for i, model_type in enumerate(df['type']):
            if model_type == 'reasoning':
                bars1[i].set_edgecolor('red')
                bars2[i].set_edgecolor('red')
                bars3[i].set_edgecolor('red')
                bars4[i].set_edgecolor('red')
                bars1[i].set_linewidth(3)
                bars2[i].set_linewidth(3)
                bars3[i].set_linewidth(3)
                bars4[i].set_linewidth(3)
        
        ax1.set_xlabel('Models')
        ax1.set_ylabel('Pass@1 Rate (%)')
        ax1.set_title('Model Performance by Difficulty Level')
        ax1.set_xticks(x_pos)
        ax1.set_xticklabels([name.split('-')[0] for name in df['model']], rotation=45)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Reasoning vs Non-Reasoning comparison
        analysis = self.analyze_difficulty_patterns(df)
        
        difficulties = ['Easy', 'Medium', 'Hard', 'Overall']
        reasoning_scores = [analysis['reasoning_avg'][d.lower()] for d in difficulties]
        non_reasoning_scores = [analysis['non_reasoning_avg'][d.lower()] for d in difficulties]
        
        x = np.arange(len(difficulties))
        width = 0.35
        
        bars1 = ax2.bar(x - width/2, non_reasoning_scores, width, 
                       label='Non-Reasoning Models', alpha=0.7, color='lightblue')
        bars2 = ax2.bar(x + width/2, reasoning_scores, width, 
                       label='Reasoning Models', alpha=0.7, color='orange')
        
        # Add improvement labels
        for i, (nr_score, r_score) in enumerate(zip(non_reasoning_scores, reasoning_scores)):
            improvement = r_score / nr_score
            ax2.text(i, max(nr_score, r_score) + 2, f'{improvement:.1f}x', 
                    ha='center', va='bottom', fontweight='bold')
        
        ax2.set_xlabel('Difficulty Level')
        ax2.set_ylabel('Average Pass@1 Rate (%)')
        ax2.set_title('Reasoning vs Non-Reasoning Models')
        ax2.set_xticks(x)
        ax2.set_xticklabels(difficulties)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Initialize analyzer and create visualizations
analyzer = ReasoningAnalyzer()
df = analyzer.create_performance_dataframe()

print("Model Performance Data:")
print(df.to_string(index=False))

# Analyze patterns
difficulty_analysis = analyzer.analyze_difficulty_patterns(df)
print("\nDifficulty Analysis:")
print(json.dumps(difficulty_analysis, indent=2))

# Create visualization
analyzer.create_difficulty_visualization(df)

## 3. Topic-Wise Performance Analysis

### Understanding Where Reasoning Helps Most

In [ ]:
class TopicAnalyzer:
    """Analyze performance across different algorithmic topics"""
    
    def __init__(self, topic_data: Dict):
        self.topic_data = topic_data
        
    def create_topic_dataframe(self) -> pd.DataFrame:
        """Convert topic data to DataFrame"""
        data = []
        for topic, scores in self.topic_data.items():
            for model, score in scores.items():
                model_type = 'reasoning' if model in ['DeepSeek-R1', 'QwQ-Plus'] else 'non_reasoning'
                data.append({
                    'topic': topic,
                    'model': model,
                    'type': model_type,
                    'score': score
                })
        return pd.DataFrame(data)
    
    def calculate_reasoning_advantage(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calculate reasoning advantage for each topic"""
        results = []
        
        for topic in df['topic'].unique():
            topic_data = df[df['topic'] == topic]
            
            reasoning_scores = topic_data[topic_data['type'] == 'reasoning']['score']
            non_reasoning_scores = topic_data[topic_data['type'] == 'non_reasoning']['score']
            
            reasoning_avg = reasoning_scores.mean()
            non_reasoning_avg = non_reasoning_scores.mean()
            
            advantage = reasoning_avg / non_reasoning_avg if non_reasoning_avg > 0 else 0
            absolute_diff = reasoning_avg - non_reasoning_avg
            
            # Calculate variance (consistency)
            reasoning_var = reasoning_scores.var()
            non_reasoning_var = non_reasoning_scores.var()
            
            results.append({
                'topic': topic,
                'reasoning_avg': reasoning_avg,
                'non_reasoning_avg': non_reasoning_avg,
                'advantage_ratio': advantage,
                'absolute_difference': absolute_diff,
                'reasoning_variance': reasoning_var,
                'non_reasoning_variance': non_reasoning_var,
                'consistency_improvement': non_reasoning_var / reasoning_var if reasoning_var > 0 else 1
            })
        
        return pd.DataFrame(results).sort_values('advantage_ratio', ascending=False)
    
    def categorize_topics_by_reasoning_benefit(self, advantage_df: pd.DataFrame) -> Dict:
        """Categorize topics by how much reasoning helps"""
        categories = {
            'high_benefit': [],      # >2x improvement
            'medium_benefit': [],    # 1.5-2x improvement  
            'low_benefit': [],       # 1.2-1.5x improvement
            'minimal_benefit': []    # <1.2x improvement
        }
        
        for _, row in advantage_df.iterrows():
            ratio = row['advantage_ratio']
            topic = row['topic']
            
            if ratio >= 2.0:
                categories['high_benefit'].append(topic)
            elif ratio >= 1.5:
                categories['medium_benefit'].append(topic)
            elif ratio >= 1.2:
                categories['low_benefit'].append(topic)
            else:
                categories['minimal_benefit'].append(topic)
        
        return categories
    
    def create_topic_heatmap(self, df: pd.DataFrame):
        """Create heatmap of topic performance"""
        # Pivot data for heatmap
        pivot_df = df.pivot(index='topic', columns='model', values='score')
        
        # Reorder columns to group reasoning vs non-reasoning
        reasoning_models = ['DeepSeek-R1', 'QwQ-Plus']
        non_reasoning_models = ['GPT-4o', 'DeepSeek-V3', 'Claude-3.7']
        column_order = non_reasoning_models + reasoning_models
        
        pivot_df = pivot_df[column_order]
        
        plt.figure(figsize=(12, 8))
        
        # Create heatmap
        sns.heatmap(pivot_df, annot=True, fmt='.1f', cmap='YlOrRd', 
                   cbar_kws={'label': 'Pass Rate (%)'}, 
                   linewidths=0.5)
        
        # Add vertical line to separate reasoning vs non-reasoning
        plt.axvline(x=len(non_reasoning_models), color='blue', linewidth=3, alpha=0.7)
        
        plt.title('Model Performance Across Algorithmic Topics', fontsize=16, pad=20)
        plt.xlabel('Models', fontsize=12)
        plt.ylabel('Topics', fontsize=12)
        
        # Add annotations
        plt.text(len(non_reasoning_models)/2, -0.5, 'Non-Reasoning Models', 
                ha='center', va='top', fontweight='bold', color='darkred')
        plt.text(len(non_reasoning_models) + len(reasoning_models)/2, -0.5, 'Reasoning Models', 
                ha='center', va='top', fontweight='bold', color='darkgreen')
        
        plt.tight_layout()
        plt.show()
    
    def create_advantage_analysis_plot(self, advantage_df: pd.DataFrame):
        """Create visualization of reasoning advantages"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Plot 1: Advantage ratio by topic
        topics = advantage_df['topic']
        ratios = advantage_df['advantage_ratio']
        
        bars = ax1.barh(topics, ratios, alpha=0.7)
        
        # Color bars by advantage level
        for i, ratio in enumerate(ratios):
            if ratio >= 2.0:
                bars[i].set_color('darkgreen')
            elif ratio >= 1.5:
                bars[i].set_color('green')
            elif ratio >= 1.2:
                bars[i].set_color('orange')
            else:
                bars[i].set_color('red')
        
        ax1.axvline(x=1, color='black', linestyle='--', alpha=0.5, label='No advantage')
        ax1.axvline(x=1.5, color='orange', linestyle='--', alpha=0.5, label='Medium benefit')
        ax1.axvline(x=2.0, color='green', linestyle='--', alpha=0.5, label='High benefit')
        
        ax1.set_xlabel('Reasoning Advantage Ratio')
        ax1.set_ylabel('Topics')
        ax1.set_title('Reasoning Model Advantage by Topic')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Add value labels
        for i, ratio in enumerate(ratios):
            ax1.text(ratio + 0.05, i, f'{ratio:.1f}x', 
                    va='center', fontweight='bold')
        
        # Plot 2: Absolute performance comparison
        x = np.arange(len(topics))
        width = 0.35
        
        bars1 = ax2.bar(x - width/2, advantage_df['non_reasoning_avg'], width, 
                       label='Non-Reasoning Avg', alpha=0.7, color='lightblue')
        bars2 = ax2.bar(x + width/2, advantage_df['reasoning_avg'], width, 
                       label='Reasoning Avg', alpha=0.7, color='orange')
        
        ax2.set_xlabel('Topics')
        ax2.set_ylabel('Average Pass Rate (%)')
        ax2.set_title('Absolute Performance Comparison')
        ax2.set_xticks(x)
        ax2.set_xticklabels(topics, rotation=45, ha='right')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Analyze topic-wise performance
topic_analyzer = TopicAnalyzer(analyzer.topic_data)
topic_df = topic_analyzer.create_topic_dataframe()
advantage_df = topic_analyzer.calculate_reasoning_advantage(topic_df)

print("Topic-wise Reasoning Advantage:")
print(advantage_df.to_string(index=False))

# Categorize topics
categories = topic_analyzer.categorize_topics_by_reasoning_benefit(advantage_df)
print("\nTopic Categories by Reasoning Benefit:")
for category, topics in categories.items():
    print(f"{category.upper()}: {', '.join(topics)}")

# Create visualizations
topic_analyzer.create_topic_heatmap(topic_df)
topic_analyzer.create_advantage_analysis_plot(advantage_df)

## 4. Reasoning Quality Metrics

### Measuring the Quality of Reasoning Process

In [ ]:
@dataclass
class ReasoningTrace:
    """Represents a reasoning trace from a model"""
    model_name: str
    problem_id: str
    reasoning_text: str
    solution_code: str
    is_correct: bool
    
class ReasoningQualityAnalyzer:
    """Analyze the quality of reasoning in model outputs"""
    
    def __init__(self):
        self.reasoning_indicators = {
            'step_indicators': ['step', 'first', 'next', 'then', 'finally'],
            'logical_connectors': ['because', 'since', 'therefore', 'thus', 'hence'],
            'analysis_terms': ['analyze', 'consider', 'examine', 'observe', 'notice'],
            'algorithmic_terms': ['algorithm', 'approach', 'strategy', 'method', 'technique'],
            'complexity_terms': ['time complexity', 'space complexity', 'efficient', 'optimal'],
            'verification_terms': ['check', 'verify', 'test', 'validate', 'confirm']
        }
    
    def calculate_reasoning_metrics(self, trace: ReasoningTrace) -> Dict:
        """Calculate comprehensive reasoning quality metrics"""
        text = trace.reasoning_text.lower()
        words = text.split()
        sentences = text.split('.')
        
        metrics = {
            'basic_metrics': {
                'word_count': len(words),
                'sentence_count': len(sentences),
                'avg_sentence_length': len(words) / max(len(sentences), 1)
            },
            'reasoning_indicators': {},
            'structural_quality': {},
            'overall_scores': {}
        }
        
        # Count reasoning indicators
        for category, indicators in self.reasoning_indicators.items():
            count = sum(1 for indicator in indicators if indicator in text)
            density = count / max(len(words), 1) * 100
            metrics['reasoning_indicators'][category] = {
                'count': count,
                'density': density
            }
        
        # Analyze structural quality
        metrics['structural_quality'] = {
            'has_clear_steps': self._has_clear_steps(text),
            'has_examples': self._has_examples(text),
            'has_complexity_analysis': self._has_complexity_analysis(text),
            'has_verification': self._has_verification(text),
            'coherence_score': self._calculate_coherence_score(text)
        }
        
        # Calculate overall scores
        metrics['overall_scores'] = {
            'reasoning_density': sum(cat['density'] for cat in metrics['reasoning_indicators'].values()),
            'structural_score': sum(metrics['structural_quality'].values()) / len(metrics['structural_quality']) * 100,
            'completeness_score': self._calculate_completeness_score(trace),
            'correctness_bonus': 20 if trace.is_correct else 0
        }
        
        # Final quality score
        metrics['overall_scores']['total_quality'] = (
            metrics['overall_scores']['reasoning_density'] * 0.3 +
            metrics['overall_scores']['structural_score'] * 0.3 +
            metrics['overall_scores']['completeness_score'] * 0.2 +
            metrics['overall_scores']['correctness_bonus'] * 0.2
        )
        
        return metrics
    
    def _has_clear_steps(self, text: str) -> bool:
        """Check if text has clear step-by-step structure"""
        step_patterns = ['step 1', 'step 2', '1.', '2.', 'first,', 'second,', 'next,']
        return any(pattern in text for pattern in step_patterns)
    
    def _has_examples(self, text: str) -> bool:
        """Check if text includes examples"""
        example_patterns = ['example', 'for instance', 'consider', 'suppose']
        return any(pattern in text for pattern in example_patterns)
    
    def _has_complexity_analysis(self, text: str) -> bool:
        """Check if text includes complexity analysis"""
        complexity_patterns = ['o(', 'time complexity', 'space complexity', 'runtime']
        return any(pattern in text for pattern in complexity_patterns)
    
    def _has_verification(self, text: str) -> bool:
        """Check if text includes verification steps"""
        verification_patterns = ['verify', 'check', 'test', 'validate', 'correct']
        return any(pattern in text for pattern in verification_patterns)
    
    def _calculate_coherence_score(self, text: str) -> float:
        """Calculate coherence score based on logical flow"""
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        if len(sentences) < 2:
            return 0.5
        
        # Simple heuristic: check for transition words
        transitions = ['therefore', 'thus', 'then', 'next', 'however', 'but', 'so']
        transition_count = sum(1 for sentence in sentences 
                             for transition in transitions 
                             if transition in sentence.lower())
        
        return min(1.0, transition_count / len(sentences) * 3)
    
    def _calculate_completeness_score(self, trace: ReasoningTrace) -> float:
        """Calculate completeness of the reasoning process"""
        text = trace.reasoning_text.lower()
        
        # Check for essential components
        components = {
            'problem_understanding': any(word in text for word in ['understand', 'problem', 'need', 'goal']),
            'approach_identification': any(word in text for word in ['approach', 'method', 'algorithm', 'strategy']),
            'implementation_plan': any(word in text for word in ['implement', 'code', 'solution', 'steps']),
            'edge_cases': any(word in text for word in ['edge', 'corner', 'special', 'boundary'])
        }
        
        return sum(components.values()) / len(components) * 100
    
    def create_mock_reasoning_traces(self) -> List[ReasoningTrace]:
        """Create mock reasoning traces for demonstration"""
        traces = []
        
        # Non-reasoning model trace (shorter, less detailed)
        traces.append(ReasoningTrace(
            model_name="GPT-4o",
            problem_id="missing_number_ap",
            reasoning_text="""Looking at this problem, I need to find the missing number in an arithmetic progression. 
            I can calculate the expected sum and subtract the actual sum.""",
            solution_code="return expected_sum - actual_sum",
            is_correct=True
        ))
        
        # Reasoning model trace (longer, more detailed)
        traces.append(ReasoningTrace(
            model_name="DeepSeek-R1",
            problem_id="missing_number_ap",
            reasoning_text="""Let me analyze this step by step.
            
            Step 1: Understanding the problem
            We have an arithmetic progression with one missing element. In an AP, consecutive differences are constant.
            
            Step 2: Approach identification
            I can use the sum formula for arithmetic progression: sum = n * (first + last) / 2
            Since we have n-1 elements (one missing), the expected sum would be (n+1) * (first + last) / 2
            
            Step 3: Implementation strategy
            - Calculate expected sum of complete sequence
            - Calculate actual sum of given array
            - Return the difference
            
            Step 4: Complexity analysis
            Time complexity: O(n) for summing the array
            Space complexity: O(1) for variables
            
            Step 5: Verification
            Let me check with the example: [5,7,11,13] missing 9
            Expected sum: 5 * (5+13) / 2 = 45
            Actual sum: 5+7+11+13 = 36
            Difference: 45-36 = 9 ✓""",
            solution_code="""expected_sum = (len(arr) + 1) * (arr[0] + arr[-1]) // 2
            actual_sum = sum(arr)
            return expected_sum - actual_sum""",
            is_correct=True
        ))
        
        # Another non-reasoning trace
        traces.append(ReasoningTrace(
            model_name="Claude-3.7",
            problem_id="missing_number_ap",
            reasoning_text="""I'll iterate through the array to find where the common difference breaks.
            The common difference should be (last - first) / n.""",
            solution_code="""diff = (arr[-1] - arr[0]) // len(arr)
            for i in range(len(arr)-1):
                if arr[i+1] - arr[i] != diff:
                    return arr[i] + diff""",
            is_correct=True
        ))
        
        return traces
    
    def analyze_model_reasoning_patterns(self, traces: List[ReasoningTrace]) -> pd.DataFrame:
        """Analyze reasoning patterns across models"""
        results = []
        
        for trace in traces:
            metrics = self.calculate_reasoning_metrics(trace)
            
            result = {
                'model': trace.model_name,
                'problem': trace.problem_id,
                'word_count': metrics['basic_metrics']['word_count'],
                'reasoning_density': metrics['overall_scores']['reasoning_density'],
                'structural_score': metrics['overall_scores']['structural_score'],
                'completeness_score': metrics['overall_scores']['completeness_score'],
                'total_quality': metrics['overall_scores']['total_quality'],
                'is_correct': trace.is_correct
            }
            
            # Add individual indicator scores
            for category, data in metrics['reasoning_indicators'].items():
                result[f'{category}_density'] = data['density']
            
            results.append(result)
        
        return pd.DataFrame(results)
    
    def create_reasoning_quality_visualization(self, analysis_df: pd.DataFrame):
        """Create visualization of reasoning quality analysis"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot 1: Total quality scores
        models = analysis_df['model']
        quality_scores = analysis_df['total_quality']
        
        bars = ax1.bar(models, quality_scores, alpha=0.7)
        
        # Color by model type
        reasoning_models = ['DeepSeek-R1', 'QwQ-Plus']
        for i, model in enumerate(models):
            if model in reasoning_models:
                bars[i].set_color('orange')
            else:
                bars[i].set_color('lightblue')
        
        ax1.set_xlabel('Models')
        ax1.set_ylabel('Total Quality Score')
        ax1.set_title('Reasoning Quality Scores by Model')
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Component breakdown
        components = ['reasoning_density', 'structural_score', 'completeness_score']
        x = np.arange(len(models))
        width = 0.25
        
        for i, component in enumerate(components):
            values = analysis_df[component]
            ax2.bar(x + i*width, values, width, label=component.replace('_', ' ').title(), alpha=0.7)
        
        ax2.set_xlabel('Models')
        ax2.set_ylabel('Score')
        ax2.set_title('Quality Component Breakdown')
        ax2.set_xticks(x + width)
        ax2.set_xticklabels(models)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Reasoning indicators heatmap
        indicator_cols = [col for col in analysis_df.columns if col.endswith('_density') and 'reasoning' not in col]
        heatmap_data = analysis_df[['model'] + indicator_cols].set_index('model')
        
        im = ax3.imshow(heatmap_data.values, cmap='YlOrRd', aspect='auto')
        ax3.set_xticks(range(len(indicator_cols)))
        ax3.set_xticklabels([col.replace('_density', '').replace('_', ' ').title() for col in indicator_cols], rotation=45)
        ax3.set_yticks(range(len(models)))
        ax3.set_yticklabels(models)
        ax3.set_title('Reasoning Indicator Density')
        
        # Add colorbar
        plt.colorbar(im, ax=ax3, shrink=0.6)
        
        # Plot 4: Word count vs quality
        word_counts = analysis_df['word_count']
        quality_scores = analysis_df['total_quality']
        
        for i, (wc, qs, model) in enumerate(zip(word_counts, quality_scores, models)):
            color = 'orange' if model in reasoning_models else 'lightblue'
            ax4.scatter(wc, qs, color=color, s=200, alpha=0.7)
            ax4.annotate(model, (wc, qs), xytext=(5, 5), textcoords='offset points')
        
        ax4.set_xlabel('Word Count')
        ax4.set_ylabel('Total Quality Score')
        ax4.set_title('Reasoning Length vs Quality')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Analyze reasoning quality
quality_analyzer = ReasoningQualityAnalyzer()
mock_traces = quality_analyzer.create_mock_reasoning_traces()

print("Mock Reasoning Traces:")
for i, trace in enumerate(mock_traces):
    print(f"\nTrace {i+1} ({trace.model_name}):")
    print(f"Length: {len(trace.reasoning_text.split())} words")
    print("Reasoning:", trace.reasoning_text[:150] + "...")

# Analyze reasoning patterns
analysis_df = quality_analyzer.analyze_model_reasoning_patterns(mock_traces)
print("\nReasoning Quality Analysis:")
print(analysis_df.to_string(index=False, float_format='%.1f'))

# Create visualization
quality_analyzer.create_reasoning_quality_visualization(analysis_df)

## 5. Performance Consistency Analysis

### Understanding Model Reliability

In [ ]:
class ConsistencyAnalyzer:
    """Analyze performance consistency across different dimensions"""
    
    def __init__(self, topic_data: Dict):
        self.topic_data = topic_data
        
    def calculate_consistency_metrics(self) -> pd.DataFrame:
        """Calculate consistency metrics for each model"""
        results = []
        
        # Get all models
        all_models = set()
        for topic_scores in self.topic_data.values():
            all_models.update(topic_scores.keys())
        
        for model in all_models:
            # Get scores across all topics for this model
            scores = []
            for topic, topic_scores in self.topic_data.items():
                if model in topic_scores:
                    scores.append(topic_scores[model])
            
            if scores:
                model_type = 'reasoning' if model in ['DeepSeek-R1', 'QwQ-Plus'] else 'non_reasoning'
                
                results.append({
                    'model': model,
                    'type': model_type,
                    'mean_score': np.mean(scores),
                    'std_score': np.std(scores),
                    'min_score': np.min(scores),
                    'max_score': np.max(scores),
                    'range': np.max(scores) - np.min(scores),
                    'coefficient_of_variation': np.std(scores) / np.mean(scores) if np.mean(scores) > 0 else 0,
                    'consistency_score': 100 - (np.std(scores) / np.mean(scores) * 100) if np.mean(scores) > 0 else 0
                })
        
        return pd.DataFrame(results).sort_values('consistency_score', ascending=False)
    
    def analyze_failure_patterns(self) -> Dict:
        """Analyze where models tend to fail"""
        analysis = {
            'challenging_topics': {},
            'model_weaknesses': {},
            'topic_difficulty_ranking': []
        }
        
        # Find most challenging topics (lowest average scores)
        topic_averages = {}
        for topic, scores in self.topic_data.items():
            topic_averages[topic] = np.mean(list(scores.values()))
        
        # Sort by difficulty (lowest scores = most difficult)
        sorted_topics = sorted(topic_averages.items(), key=lambda x: x[1])
        analysis['topic_difficulty_ranking'] = sorted_topics
        
        # Identify challenging topics (bottom 25%)
        num_challenging = max(1, len(sorted_topics) // 4)
        analysis['challenging_topics'] = dict(sorted_topics[:num_challenging])
        
        # Find each model's weakest areas
        for topic, scores in self.topic_data.items():
            for model, score in scores.items():
                if model not in analysis['model_weaknesses']:
                    analysis['model_weaknesses'][model] = []
                
                # Consider it a weakness if score is below 30%
                if score < 30:
                    analysis['model_weaknesses'][model].append((topic, score))
        
        return analysis
    
    def create_consistency_visualization(self, consistency_df: pd.DataFrame, failure_analysis: Dict):
        """Create comprehensive consistency visualization"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot 1: Mean vs Standard Deviation
        reasoning_models = consistency_df[consistency_df['type'] == 'reasoning']
        non_reasoning_models = consistency_df[consistency_df['type'] == 'non_reasoning']
        
        ax1.scatter(reasoning_models['mean_score'], reasoning_models['std_score'], 
                   color='orange', s=200, alpha=0.7, label='Reasoning Models')
        ax1.scatter(non_reasoning_models['mean_score'], non_reasoning_models['std_score'], 
                   color='lightblue', s=200, alpha=0.7, label='Non-Reasoning Models')
        
        # Add model labels
        for _, row in consistency_df.iterrows():
            ax1.annotate(row['model'], (row['mean_score'], row['std_score']),
                        xytext=(5, 5), textcoords='offset points')
        
        ax1.set_xlabel('Mean Performance (%)')
        ax1.set_ylabel('Standard Deviation')
        ax1.set_title('Performance vs Consistency Trade-off')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Consistency scores
        models = consistency_df['model']
        consistency_scores = consistency_df['consistency_score']
        
        bars = ax2.bar(models, consistency_scores, alpha=0.7)
        
        # Color by model type
        reasoning_model_names = ['DeepSeek-R1', 'QwQ-Plus']
        for i, model in enumerate(models):
            if model in reasoning_model_names:
                bars[i].set_color('orange')
            else:
                bars[i].set_color('lightblue')
        
        ax2.set_xlabel('Models')
        ax2.set_ylabel('Consistency Score')
        ax2.set_title('Model Consistency Ranking')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Topic difficulty ranking
        topics, avg_scores = zip(*failure_analysis['topic_difficulty_ranking'])
        
        bars = ax3.barh(topics, avg_scores, alpha=0.7)
        
        # Color by difficulty
        for i, score in enumerate(avg_scores):
            if score < 30:
                bars[i].set_color('red')
            elif score < 50:
                bars[i].set_color('orange')
            else:
                bars[i].set_color('green')
        
        ax3.set_xlabel('Average Performance (%)')
        ax3.set_ylabel('Topics')
        ax3.set_title('Topic Difficulty Ranking')
        ax3.grid(True, alpha=0.3)
        
        # Plot 4: Model weakness heatmap
        weakness_matrix = []
        model_names = []
        topic_names = list(self.topic_data.keys())
        
        for model in consistency_df['model']:
            model_names.append(model)
            row = []
            for topic in topic_names:
                score = self.topic_data[topic].get(model, 0)
                # Convert to weakness score (higher = more weakness)
                weakness = max(0, 50 - score)  # 0 if score >= 50, else 50-score
                row.append(weakness)
            weakness_matrix.append(row)
        
        im = ax4.imshow(weakness_matrix, cmap='Reds', aspect='auto')
        ax4.set_xticks(range(len(topic_names)))
        ax4.set_xticklabels(topic_names, rotation=45, ha='right')
        ax4.set_yticks(range(len(model_names)))
        ax4.set_yticklabels(model_names)
        ax4.set_title('Model Weakness Heatmap\n(Darker = More Weakness)')
        
        plt.colorbar(im, ax=ax4, shrink=0.6)
        
        plt.tight_layout()
        plt.show()
    
    def generate_consistency_report(self, consistency_df: pd.DataFrame, failure_analysis: Dict) -> str:
        """Generate comprehensive consistency report"""
        report = f"""
# Model Consistency Analysis Report

## Overall Consistency Ranking
{consistency_df[['model', 'type', 'mean_score', 'consistency_score']].to_string(index=False)}

## Key Findings

### Most Consistent Models
1. {consistency_df.iloc[0]['model']} (Consistency Score: {consistency_df.iloc[0]['consistency_score']:.1f})
2. {consistency_df.iloc[1]['model']} (Consistency Score: {consistency_df.iloc[1]['consistency_score']:.1f})
3. {consistency_df.iloc[2]['model']} (Consistency Score: {consistency_df.iloc[2]['consistency_score']:.1f})

### Most Challenging Topics
"""
        
        for topic, avg_score in failure_analysis['challenging_topics'].items():
            report += f"- {topic}: {avg_score:.1f}% average\n"
        
        report += "\n### Model-Specific Weaknesses\n"
        for model, weaknesses in failure_analysis['model_weaknesses'].items():
            if weaknesses:
                report += f"**{model}:**\n"
                for topic, score in weaknesses:
                    report += f"  - {topic}: {score:.1f}%\n"
        
        # Calculate reasoning vs non-reasoning consistency
        reasoning_consistency = consistency_df[consistency_df['type'] == 'reasoning']['consistency_score'].mean()
        non_reasoning_consistency = consistency_df[consistency_df['type'] == 'non_reasoning']['consistency_score'].mean()
        
        report += f"""
        
## Reasoning vs Non-Reasoning Consistency
- Reasoning Models Average Consistency: {reasoning_consistency:.1f}
- Non-Reasoning Models Average Consistency: {non_reasoning_consistency:.1f}
- Consistency Advantage: {reasoning_consistency - non_reasoning_consistency:.1f} points
"""
        
        return report

# Analyze consistency
consistency_analyzer = ConsistencyAnalyzer(analyzer.topic_data)
consistency_df = consistency_analyzer.calculate_consistency_metrics()
failure_analysis = consistency_analyzer.analyze_failure_patterns()

print("Consistency Analysis Results:")
print(consistency_df.to_string(index=False, float_format='%.2f'))

print("\nFailure Pattern Analysis:")
print(f"Most challenging topics: {list(failure_analysis['challenging_topics'].keys())}")

# Create visualization
consistency_analyzer.create_consistency_visualization(consistency_df, failure_analysis)

# Generate report
consistency_report = consistency_analyzer.generate_consistency_report(consistency_df, failure_analysis)
print(consistency_report)

## 6. Practical Implementation: Building a Reasoning Evaluator

### Complete Framework for Reasoning Assessment

In [ ]:
class ReasoningEvaluator:
    """Complete framework for evaluating reasoning in code generation models"""
    
    def __init__(self):
        self.quality_analyzer = ReasoningQualityAnalyzer()
        self.evaluation_metrics = {
            'correctness_weight': 0.4,
            'reasoning_quality_weight': 0.3,
            'efficiency_weight': 0.2,
            'consistency_weight': 0.1
        }
    
    def evaluate_model_response(self, problem: Dict, response: str, 
                              test_cases: List[Dict]) -> Dict:
        """Evaluate a complete model response"""
        # Parse response into reasoning and code parts
        reasoning_part, code_part = self._parse_response(response)
        
        # Test correctness
        correctness_results = self._test_correctness(code_part, test_cases)
        
        # Analyze reasoning quality
        trace = ReasoningTrace(
            model_name="test_model",
            problem_id=problem.get('id', 'unknown'),
            reasoning_text=reasoning_part,
            solution_code=code_part,
            is_correct=correctness_results['all_passed']
        )
        
        reasoning_metrics = self.quality_analyzer.calculate_reasoning_metrics(trace)
        
        # Calculate efficiency score (based on code complexity)
        efficiency_score = self._calculate_efficiency_score(code_part)
        
        # Combine all metrics
        final_score = (
            correctness_results['pass_rate'] * self.evaluation_metrics['correctness_weight'] +
            reasoning_metrics['overall_scores']['total_quality'] * self.evaluation_metrics['reasoning_quality_weight'] +
            efficiency_score * self.evaluation_metrics['efficiency_weight']
        )
        
        return {
            'final_score': final_score,
            'correctness': correctness_results,
            'reasoning_quality': reasoning_metrics['overall_scores'],
            'efficiency': efficiency_score,
            'detailed_analysis': {
                'reasoning_indicators': reasoning_metrics['reasoning_indicators'],
                'structural_quality': reasoning_metrics['structural_quality']
            }
        }
    
    def _parse_response(self, response: str) -> Tuple[str, str]:
        """Parse response into reasoning and code parts"""
        # Look for code blocks
        import re
        code_pattern = r'```(?:python)?\s*([\s\S]*?)```'
        code_matches = re.findall(code_pattern, response)
        
        if code_matches:
            code_part = '\n'.join(code_matches)
            # Remove code blocks from reasoning
            reasoning_part = re.sub(code_pattern, '[CODE BLOCK]', response)
        else:
            # Try to find class/function definitions
            lines = response.split('\n')
            code_lines = []
            reasoning_lines = []
            
            in_code = False
            for line in lines:
                if line.strip().startswith(('class ', 'def ', 'return ', '    ')):
                    in_code = True
                    code_lines.append(line)
                elif in_code and (line.strip() == '' or line.startswith(' ')):
                    code_lines.append(line)
                else:
                    in_code = False
                    reasoning_lines.append(line)
            
            code_part = '\n'.join(code_lines)
            reasoning_part = '\n'.join(reasoning_lines)
        
        return reasoning_part.strip(), code_part.strip()
    
    def _test_correctness(self, code: str, test_cases: List[Dict]) -> Dict:
        """Test code correctness against test cases"""
        # In practice, this would execute code safely
        # For demo, simulate results
        passed = np.random.randint(len(test_cases) * 0.7, len(test_cases) + 1)
        
        return {
            'total_tests': len(test_cases),
            'passed_tests': passed,
            'pass_rate': passed / len(test_cases) * 100,
            'all_passed': passed == len(test_cases)
        }
    
    def _calculate_efficiency_score(self, code: str) -> float:
        """Calculate efficiency score based on code analysis"""
        lines = [line.strip() for line in code.split('\n') if line.strip()]
        
        # Simple heuristics for efficiency
        efficiency_indicators = {
            'avoid_nested_loops': 'for' in code and code.count('for') <= 2,
            'use_builtin_functions': any(func in code for func in ['sum(', 'max(', 'min(', 'sorted(']),
            'avoid_redundant_operations': 'len(' not in code or code.count('len(') <= 2,
            'concise_implementation': len(lines) <= 20
        }
        
        score = sum(efficiency_indicators.values()) / len(efficiency_indicators) * 100
        
        # Bonus for mathematical approaches
        if any(term in code.lower() for term in ['math.', '//', 'sum(']):
            score += 10
        
        return min(100, score)
    
    def benchmark_models(self, problems: List[Dict], 
                        model_responses: Dict[str, List[str]]) -> pd.DataFrame:
        """Benchmark multiple models on multiple problems"""
        results = []
        
        for problem in problems:
            test_cases = problem.get('test_cases', [])
            
            for model_name, responses in model_responses.items():
                if len(responses) > problems.index(problem):
                    response = responses[problems.index(problem)]
                    evaluation = self.evaluate_model_response(problem, response, test_cases)
                    
                    results.append({
                        'model': model_name,
                        'problem_id': problem.get('id', f"problem_{problems.index(problem)}"),
                        'difficulty': problem.get('difficulty', 'Unknown'),
                        'final_score': evaluation['final_score'],
                        'correctness_score': evaluation['correctness']['pass_rate'],
                        'reasoning_quality': evaluation['reasoning_quality']['total_quality'],
                        'efficiency_score': evaluation['efficiency']
                    })
        
        return pd.DataFrame(results)
    
    def create_benchmark_visualization(self, benchmark_df: pd.DataFrame):
        """Create comprehensive benchmark visualization"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot 1: Overall model ranking
        model_averages = benchmark_df.groupby('model')['final_score'].mean().sort_values(ascending=False)
        
        bars = ax1.bar(model_averages.index, model_averages.values, alpha=0.7)
        
        # Color reasoning models differently
        reasoning_models = ['DeepSeek-R1', 'QwQ-Plus']
        for i, model in enumerate(model_averages.index):
            if any(rm in model for rm in reasoning_models):
                bars[i].set_color('orange')
            else:
                bars[i].set_color('lightblue')
        
        ax1.set_xlabel('Models')
        ax1.set_ylabel('Average Final Score')
        ax1.set_title('Overall Model Ranking')
        ax1.tick_params(axis='x', rotation=45)
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Score components breakdown
        components = ['correctness_score', 'reasoning_quality', 'efficiency_score']
        model_names = benchmark_df['model'].unique()
        x = np.arange(len(model_names))
        width = 0.25
        
        for i, component in enumerate(components):
            component_averages = benchmark_df.groupby('model')[component].mean()
            values = [component_averages.get(model, 0) for model in model_names]
            ax2.bar(x + i*width, values, width, label=component.replace('_', ' ').title(), alpha=0.7)
        
        ax2.set_xlabel('Models')
        ax2.set_ylabel('Score')
        ax2.set_title('Score Components Breakdown')
        ax2.set_xticks(x + width)
        ax2.set_xticklabels(model_names, rotation=45)
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Performance by difficulty
        difficulty_performance = benchmark_df.groupby(['difficulty', 'model'])['final_score'].mean().unstack()
        difficulty_performance.plot(kind='bar', ax=ax3, alpha=0.7)
        
        ax3.set_xlabel('Difficulty Level')
        ax3.set_ylabel('Average Final Score')
        ax3.set_title('Performance by Difficulty')
        ax3.legend(title='Models', bbox_to_anchor=(1.05, 1), loc='upper left')
        ax3.grid(True, alpha=0.3)
        
        # Plot 4: Correctness vs Reasoning Quality scatter
        for model in benchmark_df['model'].unique():
            model_data = benchmark_df[benchmark_df['model'] == model]
            color = 'orange' if any(rm in model for rm in reasoning_models) else 'lightblue'
            ax4.scatter(model_data['correctness_score'], model_data['reasoning_quality'], 
                       alpha=0.6, label=model, s=50, color=color)
        
        ax4.set_xlabel('Correctness Score')
        ax4.set_ylabel('Reasoning Quality Score')
        ax4.set_title('Correctness vs Reasoning Quality')
        ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Create demo benchmark
evaluator = ReasoningEvaluator()

# Mock problems and responses
demo_problems = [
    {
        'id': 'missing_number',
        'difficulty': 'Easy',
        'test_cases': [{'input': {'arr': [5, 7, 11, 13]}, 'output': 9}] * 5
    },
    {
        'id': 'binary_search',
        'difficulty': 'Medium', 
        'test_cases': [{'input': {'arr': [1, 2, 3, 4, 5], 'target': 3}, 'output': 2}] * 8
    }
]

demo_responses = {
    'GPT-4o': [
        "Looking at this problem, I need to find the missing number. I'll use the sum formula. ```python\nclass Solution:\n    def missingNumber(self, arr):\n        expected = (len(arr) + 1) * (arr[0] + arr[-1]) // 2\n        return expected - sum(arr)\n```",
        "I need to implement binary search. ```python\nclass Solution:\n    def search(self, arr, target):\n        left, right = 0, len(arr) - 1\n        while left <= right:\n            mid = (left + right) // 2\n            if arr[mid] == target: return mid\n            elif arr[mid] < target: left = mid + 1\n            else: right = mid - 1\n        return -1\n```"
    ],
    'DeepSeek-R1': [
        """Let me think through this step by step.
        
        Step 1: Understanding the problem
        We have an arithmetic progression with one missing element. I need to find that element.
        
        Step 2: Approach analysis
        I can use the sum formula for arithmetic progressions. The sum of n terms is n*(first+last)/2.
        Since we're missing one element, I need to calculate what the complete sum should be.
        
        Step 3: Implementation
        ```python
        class Solution:
            def missingNumber(self, arr: List[int]) -> int:
                n = len(arr)
                # Expected sum of complete sequence (n+1 elements)
                expected_sum = (n + 1) * (arr[0] + arr[-1]) // 2
                actual_sum = sum(arr)
                return expected_sum - actual_sum
        ```
        
        Step 4: Complexity Analysis
        Time: O(n) for computing sum
        Space: O(1) constant space
        
        Step 5: Verification
        For [5,7,11,13]: expected_sum = 5*(5+13)/2 = 45, actual = 36, missing = 9 ✓""",
        
        """I need to implement binary search with careful analysis.
        
        Step 1: Algorithm choice
        Binary search is optimal for sorted arrays, giving O(log n) time complexity.
        
        Step 2: Implementation strategy
        - Initialize left and right pointers
        - Use iterative approach to avoid recursion overhead
        - Handle edge cases properly
        
        ```python
        class Solution:
            def search(self, nums: List[int], target: int) -> int:
                left, right = 0, len(nums) - 1
                
                while left <= right:
                    mid = left + (right - left) // 2  # Avoid overflow
                    
                    if nums[mid] == target:
                        return mid
                    elif nums[mid] < target:
                        left = mid + 1
                    else:
                        right = mid - 1
                
                return -1
        ```
        
        Time: O(log n), Space: O(1)"""
    ]
}

# Run benchmark
benchmark_results = evaluator.benchmark_models(demo_problems, demo_responses)

print("Benchmark Results:")
print(benchmark_results.to_string(index=False, float_format='%.1f'))

# Create visualization
evaluator.create_benchmark_visualization(benchmark_results)

# Summary statistics
model_summary = benchmark_results.groupby('model').agg({
    'final_score': ['mean', 'std'],
    'correctness_score': 'mean',
    'reasoning_quality': 'mean',
    'efficiency_score': 'mean'
}).round(1)

print("\nModel Summary Statistics:")
print(model_summary)

## 7. Key Takeaways and Best Practices

### Critical Insights from the Paper:

1. **Reasoning Models Dominate Hard Problems**: 2.5x improvement on hard problems vs. non-reasoning models
2. **Topic-Specific Advantages**: Reasoning helps most in DP, Binary Search, Tree problems
3. **Consistency Benefits**: Reasoning models show lower variance across different topics
4. **Quality-Performance Correlation**: Longer, more detailed reasoning correlates with better results

### Performance Patterns:

**Where Reasoning Helps Most:**
- **Dynamic Programming**: 2.4x improvement (70.2% vs 15.8%)
- **Binary Search**: 2.7x improvement (73.1% vs 23.1%) 
- **Tree Problems**: 4.0x improvement (72.7% vs 18.2%)

**Where Reasoning Helps Less:**
- **Simulation**: Similar performance (63-84%)
- **String Processing**: Moderate improvement
- **Basic Array Operations**: Smaller gaps

### Implementation Best Practices:

1. **Evaluation Framework**:
   - Combine correctness, reasoning quality, and consistency metrics
   - Weight correctness heavily (40%) but consider reasoning quality (30%)
   - Track performance across different problem types

2. **Reasoning Quality Metrics**:
   - Step-by-step indicators
   - Logical connectors and transitions
   - Complexity analysis inclusion
   - Verification and examples

3. **Model Selection Guidelines**:
   - Use reasoning models for complex algorithmic problems
   - Consider non-reasoning models for simple, pattern-based tasks
   - Factor in consistency requirements for production use

### Research Implications:

1. **Chain-of-Thought is Essential**: For competitive programming, explicit reasoning significantly improves performance
2. **Problem Type Matters**: The benefit of reasoning varies dramatically by algorithmic domain
3. **Consistency vs Peak Performance**: Reasoning models offer both higher peaks and better consistency
4. **Quality Metrics**: Traditional accuracy alone is insufficient - reasoning quality matters

### Future Directions:

1. **Adaptive Reasoning**: Models that can decide when to use detailed reasoning vs. direct solutions
2. **Domain-Specific Reasoning**: Specialized reasoning patterns for different algorithmic types
3. **Efficiency-Reasoning Trade-offs**: Balancing reasoning depth with inference speed
4. **Interactive Reasoning**: Models that can refine their reasoning based on feedback